In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../')

from glob import glob

from pynlpl.formats import folia
# from folia import main as folia

In [5]:
import os
import json_lines

In [91]:
semantic_train_path = '../corpus/Token/semantic_train.txt'
semantic_valid_path = '../corpus/Token/semantic_dev.txt'
semantic_test_path = '../corpus/Token/semantic_test.txt'

In [7]:
project_folder='/home/aorus/workspaces/simge/Master_Thesis'

In [1]:
semantic_labels = ['group_clash', 'demonst', 'ind_act', 'arm_mil', 'elec_pol', 'other']

In [139]:
def prepare_semantic_data(semantic_path):
    art_id = 1
    sent_num = 1
    articles = []
    art = {
        'id': 'a.' + str(art_id),
        'sentences': []
    }
    sent = {
        'sent_num': str(art_id) + '-' + str(sent_num),
        'sentence': [],
        'label': []
    }

    with open(semantic_path, 'r', encoding='utf-8') as f:

        lines = f.readlines()

        for i, line in enumerate(lines):
            line_ = line.strip().split('\t')

            if (line_[0] == '' and (i != len(lines)-1)) or (i==0 and line_[0]=='SAMPLE_START'):
                continue

            elif line_[0] == 'SAMPLE_START' or i == len(lines)-1:
                # add the last sentence
                unique_labels = list({label[2:] for label in sent['label'] if label != 'O'})
                sent['label'] = ['O'] if not unique_labels else unique_labels
                art['sentences'].append(sent)
                articles.append(art)
                art_id = art_id + 1
                sent_num = 1

                art = {
                    'id': 'a.' + str(art_id),
                    'sentences': []
                }
                sent = {
                    'sent_num': str(art_id) + '-' + str(sent_num),
                    'sentence': [],
                    'label': []
                }

            else:
                if line_[0] == '[SEP]':
                    unique_labels = list({label[2:] for label in sent['label'] if label != 'O'})
                    sent['label'] = ['O'] if not unique_labels else unique_labels
                    art['sentences'].append(sent)
                    sent_num = sent_num + 1
                    sent = {
                        'sent_num': str(art_id) + '-' + str(sent_num),
                        'sentence': [],
                        'label': []
                    }

                else:
                    sent['sentence'].append(line_[0])
                    sent['label'].append(line_[1])

    return articles

In [140]:
train_articles = prepare_semantic_data(semantic_train_path)
valid_articles = prepare_semantic_data(semantic_valid_path)
test_articles = prepare_semantic_data(semantic_test_path)

In [142]:
train_articles[0]

{'id': 'a.1',
 'sentences': [{'sent_num': '1-1',
   'sentence': ['THALASSERY',
    ':',
    'End',
    'political',
    'killings',
    ':',
    'Rajnath',
    'September',
    '27',
    ',',
    '2014',
    '00:00',
    'IST',
    'Says',
    'decision',
    'on',
    'CBI',
    'probe',
    'into',
    'Manoj',
    'murder',
    'soon',
    'Union',
    'Home',
    'Minister',
    'Rajnath',
    'Singh',
    'has',
    'said',
    'that',
    'a',
    'decision',
    'on',
    'the',
    'probe',
    'by',
    'the',
    'Central',
    'Bureau',
    'of',
    'Investigation',
    '(',
    'CBI',
    ')',
    'into',
    'the',
    'murder',
    'of',
    'Rashtriya',
    'Swayamsevak',
    'Sangh',
    '(',
    'RSS',
    ')',
    'functionary',
    'E.',
    'Manoj',
    'will',
    'be',
    'taken',
    'soon',
    '.'],
   'label': ['group_clash']},
  {'sent_num': '1-2',
   'sentence': ['Addressing',
    'members',
    'of',
    'the',
    'families',
    'of',
    'Bharatiya',
 

In [144]:
all_semantic = {
    'train': train_articles,
    'valid': valid_articles,
    'test': test_articles
}

In [145]:
import json

In [147]:
with open('./Data/semantic_all_data', 'w') as jf:
    json.dump(all_semantic, jf)